In [ ]:
%pip install transformers 
%pip install sklearn
%pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.3 MB/s 
     |████████████████████████████████| 596 kB 38.3 MB/s 
     |████████████████████████████████| 6.6 MB 46.1 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.1 MB/s 
     |████████████████████████████████| 212 kB 49.3 MB/s 
     |████████████████████████████████| 141 kB 41.7 MB/s 
     |████████████████████████████████| 115 kB 49.3 MB/s 
     |████████████████████████████████| 127 kB 54.6 MB/s 
  Attempting uninstall: urllib3
  

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments,DataCollatorWithPadding
from datasets import load_dataset, load_metric,Dataset
import torch
import numpy as np
from utils import AssessData
from datasets import load_dataset
from utils import PrepareCorpus,AssessData
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset
import json
from re import template
from sklearn.model_selection import KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
parentdir = "/content/drive/MyDrive/ML/Datasets/20News"

train =parentdir+"/20news-bydate"+"/20news-bydate-train"
test = parentdir+"/20news-bydate"+ "/20news-bydate-test"

In [ ]:
id2labels = {'talk.politics.mideast': 0, 'rec.autos': 1, 'comp.sys.mac.hardware': 2, 'alt.atheism': 3, 'rec.sport.baseball': 4, 'comp.os.ms-windows.misc': 5, 'rec.sport.hockey': 6, 'sci.crypt': 7, 'sci.med': 8, 'talk.politics.misc': 9, 'rec.motorcycles': 10, 'comp.windows.x': 11, 'comp.graphics': 12, 'comp.sys.ibm.pc.hardware': 13, 'sci.electronics': 14, 'talk.politics.guns': 15, 'sci.space': 16, 'soc.religion.christian': 17, 'misc.forsale': 18, 'talk.religion.misc': 19}
id2classes = {'talk.politics.mideast': 0, 'rec.autos': 4, 'comp.sys.mac.hardware': 2, 'alt.atheism': 5, 'rec.sport.baseball': 1, 'comp.os.ms-windows.misc': 2, 'rec.sport.hockey': 1, 'sci.crypt': 3, 'sci.med': 3, 'talk.politics.misc': 0, 'rec.motorcycles': 4, 'comp.windows.x': 2, 'comp.graphics': 2, 'comp.sys.ibm.pc.hardware': 2, 'sci.electronics': 3, 'talk.politics.guns': 0, 'sci.space': 3, 'soc.religion.christian': 5, 'misc.forsale': 6, 'talk.religion.misc': 5}

max_input_length = 300
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

number = range(50,150,50)
_per_segment = range(200,400,100)

overlap = {"side":"both", "number": 50}

f1_score = load_metric("f1"); precision = load_metric("precision"); recall = load_metric("recall")

def tokenize(batch):
    return bert_tokenizer(batch['text'], truncation=True, max_length=max_input_length,padding = True)

def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return {"f1" :f1_score.compute(predictions = predictions, references = label, average = 'weighted'),
            "precision" : precision.compute(predictions = predictions, references = label, average = 'weighted'),
            "recall": recall.compute(predictions = predictions, references = label, average = 'weighted')}

def hp_space(trial):
  return {"per_device_train_batch_size": trial.suggest_discrete_uniform("per_device_train_batch_size", 8,32,8)
        ,"learning_rate": trial.suggest_float("learning_rate", 0.00001,0.00005, log = True)
        ,"num_train_epochs": trial.suggest_int("num_train_epochs",1,10)}

def bert_init():
  return BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 7)

def id_2_labels(x,adict):
  """converts labels/classes into a number using a dictionary"""
  return adict[x]

def _indexing(alist):
  id = 0; indexed = {}
  for x in alist:
    for y in x:
      id +=1
      indexed[id] = y
  return indexed
  

In [ ]:
news_group_train = PrepareCorpus(train)
news_group_test = PrepareCorpus(test)

In [ ]:
news_group_test = news_group_test._prep()       #Keys are labels, values are texts
news_group_train = news_group_train._prep()

SUBCATEGORY MAPPING

In [ ]:
#select only politics
id_politics = {'talk.politics.misc': 0, 'talk.politics.mideast': 1, 'talk.politics.guns': 2}
id_religion = {'alt.atheism': 0, 'soc.religion.christian' : 1, 'talk.religion.misc': 2}
id_sport = {'rec.sport.baseball': 0, 'rec.sport.hockey': 1}
id_autos = {'rec.motorcycles': 0, 'rec.autos': 1,}
id_sci = {'sci.crypt': 0, 'sci.med': 1, 'sci.space': 2}
id_comp = {'comp.sys.mac.hardware': 0,  'comp.os.ms-windows.misc': 1, 'comp.windows.x': 2, 'comp.graphics': 3, 'comp.sys.ibm.pc.hardware': 4, 'sci.electronics': 5}
id_sale = {'misc.forsale': 0}

In [ ]:
#Selecting id_autos documents only

temp_train = []; temp_test = []
temp_news_group_train = {}; temp_news_group_test = {}

for item in id_autos.keys():
  temp_news_group_train[item] = news_group_train.get(item)
  temp_news_group_test[item] = news_group_test.get(item)

temp_train = _indexing(temp_news_group_train.values())
temp_test =  _indexing(temp_news_group_test.values())

temp_train_corpus = AssessData(temp_train, temp_news_group_train)
temp_test_corpus = AssessData(temp_test, temp_news_group_test)

In [ ]:
type(temp_train_corpus)

utils.AssessData

In [ ]:
temp_train_index_to_label = temp_train_corpus._index_to_label() 
temp_test_index_to_label = temp_test_corpus._index_to_label()

In [ ]:
set(temp_train_index_to_label.values())

{'rec.autos', 'rec.motorcycles'}

In [ ]:
temp_train_index_to_label = {i:id_2_labels(x, id_autos) for i,x in temp_train_index_to_label.items()}
temp_test_index_to_label = {i:id_2_labels(x, id_autos) for i,x in temp_test_index_to_label.items()}

In [ ]:
temp_train_set = temp_train_corpus._chunk(200, overlap=overlap)
temp_train_index, temp_train_text = zip(*temp_train_set)

temp_test_set = temp_test_corpus._chunk(200, overlap=overlap)
test_index, test_text = zip(*temp_test_set)

In [ ]:
train_index = {a:x-1 for a,x in enumerate(temp_train_index)}
train_text = {a:x for a,x in enumerate(temp_train_text)}

In [ ]:
test_index = {a:x-1 for a,x in enumerate(test_index)}
test_text = {a:x for a,x in enumerate(test_text)}

In [ ]:
train_chunked_label = {i:id_2_labels(x,temp_train_index_to_label) for i,x in train_index.items()}
test_chunked_label = {i:id_2_labels(x,temp_test_index_to_label) for i,x in test_index.items()}

In [ ]:
train_chunked_label

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 0,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 0,
 117: 0,
 118: 0,
 119: 0,
 120: 0,
 121: 0,
 122: 0,
 12

In [ ]:
autos_train_set = {"train" : {"text": train_text.values(), 'doc_id': train_index.values(), 'labels': train_chunked_label.values()}}
autos_test_set =  {"test" : {"text": test_text.values(), 'doc_id': test_index.values(), 'labels': test_chunked_label.values()}}

In [ ]:
import pandas as pd

In [ ]:
a = pd.DataFrame(autos_train_set['train'])
b = pd.DataFrame(autos_test_set['test'])

In [ ]:
a

,text,doc_id,labels
0,From: prange@nickel.ucs.indiana.edu (Henry Pra...,0,0
1,ndiana University\n Lines: 20\n \n In article ...,0,0
2,"ss here, of does anyone else know\n >what I'm ...",0,0
3,.Levine@hydro.on.ca\n > ...,0,0
4,ionals.\n \n Henry Prange biker/professional ...,0,0
...,...,...,...
10299,Distribution: usa\n Lines: 15\n \n In article...,1191,1
10300,and I are in the market to buy our first car\n...,1191,1
10301,for it?\n \n Several company publish periodica...,1191,1
10302,price guide. You can find\n these price guide...,1191,1


In [ ]:
a['text'] = a['text'].astype(str)
a['doc_id'] = a['doc_id'].astype(int)
a['labels'] = a['labels'].astype(int)

b['text'] = a['text'].astype(str)
b['doc_id'] = a['doc_id'].astype(int)
b['labels'] = a['labels'].astype(int)

In [ ]:
autos_train_set = Dataset.from_pandas(a)
autos_test_set = Dataset.from_pandas(b)

In [ ]:
autos_train_set = autos_train_set.map(tokenize)
autos_test_set = autos_test_set.map(tokenize)

autos_train_set = autos_train_set.shuffle(seed = 24)
autos_test_set = autos_test_set.shuffle(seed = 24)

  0%|          | 0/10304 [00:00<?, ?ex/s]

  0%|          | 0/6454 [00:00<?, ?ex/s]

In [ ]:
autos_test_set

Dataset({
    features: ['text', 'doc_id', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6454
})

In [ ]:
fold = KFold(n_splits = 5)
this_set = fold.split(autos_train_set['text'])

frac_fold_train = []; frac_fold_eval = []

for i,j in this_set:
  frac_fold_train.append(autos_train_set.select(i))
  frac_fold_eval.append(autos_train_set.select(j))

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer, max_length= 300)

bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = len(set(a['labels'])))

#default training arguments 
training_args = TrainingArguments(output_dir="./frac_results", learning_rate=2e-5, per_device_train_batch_size=16, per_device_eval_batch_size=16, num_train_epochs=5,weight_decay=0.01)
results = []

#fine-tune
trainer = Trainer(model = bert_model, 
                args = training_args,
                train_dataset= frac_fold_train[0],
                eval_dataset= frac_fold_eval[0],
                tokenizer= bert_tokenizer,
                data_collator=data_collator,
                compute_metrics= compute_metrics)

trainer.train()

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

Step,Training Loss


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: doc_id, text. If doc_id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2061
  Batch size = 16
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2340: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "


Trainer is attempting to log a value of "{'f1': 0.8745675673932726}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8824387916938107}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8753032508491024}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'epoch': 5.0,
 'eval_f1': {'f1': 0.8745675673932726},
 'eval_loss': 0.3531061112880707,
 'eval_precision': {'precision': 0.8824387916938107},
 'eval_recall': {'recall': 0.8753032508491024},
 'eval_runtime': 7.699,
 'eval_samples_per_second': 267.696,
 'eval_steps_per_second': 16.755}